In [79]:
import os
import json
import pandas as pd

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [80]:
from collectors.etherscanapi import EtherscanAPI, get_timestamp_days_ago
from collectors.defillamaapi import DefillamaAPI

In [81]:
ETHERSCAN_API_KEY = os.getenv("ETHERSCAN_API_KEY")
etherscan_api = EtherscanAPI(ETHERSCAN_API_KEY)

pd.set_option('display.max_columns', None)

## Retrieve Transaction Data for Fusion Settlement
0xA88800CD213dA5Ae406ce248380802BD53b47647 is the settlement contract address. We can use the contract address to retrieve the transaction data for the settlement contract.

In [82]:
timestamp = get_timestamp_days_ago(1)
block_number = etherscan_api.get_block_number_by_timestamp(timestamp)
block_number # block number to for start date of transaction query

'17048915'

In [83]:
address = "0xA88800CD213dA5Ae406ce248380802BD53b47647"
transactions = etherscan_api.get_transactions(address, block_number)
df = etherscan_api.process_transactions(transactions)
df = df.query("isError == '0'")
df.head()

,blockNumber,timeStamp,hash,nonce,blockHash,transactionIndex,from,to,value,gas,gasPrice,isError,txreceipt_status,input,contractAddress,cumulativeGasUsed,gasUsed,confirmations,methodId,functionName
0,17048931,1681518767,0x05eaf873b03fd34df0b5a62bbb88e2ef6fec97837147...,26503,0x77e1172245d5b346568bd60c5aa6245a5ffdea7a05fc...,74,0x9108813f22637385228a1c621c1904bbbc50dc25,0xa88800cd213da5ae406ce248380802bd53b47647,0,508565,23544265231,0,1,0x0965d04b000000000000000000000000000000000000...,,3955432,240453,7005,0x0965d04b,settleOrders(bytes data)
1,17048932,1681518779,0xddd47b8c66f61376e14aba9c05e3a7b1aea53e8bf44f...,6097,0x90e4277c73d1e80a05eff9741a0af7f00d7ea307e978...,5,0xcfa62f77920d6383be12c91c71bd403599e1116f,0xa88800cd213da5ae406ce248380802bd53b47647,0,667536,33015544778,0,1,0x0965d04b000000000000000000000000000000000000...,,793292,349212,7004,0x0965d04b,settleOrders(bytes data)
2,17048949,1681518983,0x6c83f6076738c8c73daff43c576f57da289d0a798c93...,26684,0x0722b50c4f7539e2539d0ef31f0934a775ea279ac0fe...,6,0xc6c7565644ea1893ad29182f7b6961aab7edfed0,0xa88800cd213da5ae406ce248380802bd53b47647,0,1500000,35557182698,0,1,0x0965d04b000000000000000000000000000000000000...,,1213514,298376,6987,0x0965d04b,settleOrders(bytes data)
4,17048967,1681519199,0xde912cdb7220293b1874623560a48e19debd35529337...,5071,0x322da2b6d1966ae5d7de4a5f0df79e17b2591c9a3d8c...,0,0xee230dd7519bc5d0c9899e8704ffdc80560e8509,0xa88800cd213da5ae406ce248380802bd53b47647,0,2000000,31458663012,0,1,0x0965d04b000000000000000000000000000000000000...,,716163,716163,6969,0x0965d04b,settleOrders(bytes data)
5,17049003,1681519679,0x732a2f175ff9d340039ba4b62422c4a9b21d6e36dc5f...,6098,0x81802f635edf069ef1fef18bd47353e126f34b68409c...,19,0xcfa62f77920d6383be12c91c71bd403599e1116f,0xa88800cd213da5ae406ce248380802bd53b47647,0,584404,34324549950,0,1,0x0965d04b000000000000000000000000000000000000...,,2967899,306326,6933,0x0965d04b,settleOrders(bytes data)


## Retrieve Internal Transaction Data

In [84]:
# each transaction has internal erc20 transfers that need to be extracted from the logs
# demo with just 1 for now:

receipt = etherscan_api.get_transaction_receipt(df.loc[0].hash)
logs = etherscan_api.process_logs(receipt)
logs

,address,data,blockNumber,transactionHash,transactionIndex,blockHash,logIndex,removed,from_address,to_address,transaction_type
0,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x00000000000000000000000000000000000000000000...,0x1042563,0x05eaf873b03fd34df0b5a62bbb88e2ef6fec97837147...,0x4a,0x77e1172245d5b346568bd60c5aa6245a5ffdea7a05fc...,0x3f,False,0xe7c6a2767df626bca33e4e86b4f5c0c4fb4cff00,0xe789c5566b53546d46a0af48a4bd3f062d1fefd1,transfer
1,0x853d955acef822db058eb8505911ed77f175b99e,0x00000000000000000000000000000000000000000000...,0x1042563,0x05eaf873b03fd34df0b5a62bbb88e2ef6fec97837147...,0x4a,0x77e1172245d5b346568bd60c5aa6245a5ffdea7a05fc...,0x40,False,0x9a834b70c07c81a9fcd6f22e842bf002fbffbe4d,0xe789c5566b53546d46a0af48a4bd3f062d1fefd1,transfer
2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x00000000000000000000000000000000000000000000...,0x1042563,0x05eaf873b03fd34df0b5a62bbb88e2ef6fec97837147...,0x4a,0x77e1172245d5b346568bd60c5aa6245a5ffdea7a05fc...,0x41,False,0xe789c5566b53546d46a0af48a4bd3f062d1fefd1,0x9a834b70c07c81a9fcd6f22e842bf002fbffbe4d,transfer
3,0x9a834b70c07c81a9fcd6f22e842bf002fbffbe4d,0xffffffffffffffffffffffffffffffffffffffffffff...,0x1042563,0x05eaf873b03fd34df0b5a62bbb88e2ef6fec97837147...,0x4a,0x77e1172245d5b346568bd60c5aa6245a5ffdea7a05fc...,0x42,False,0x1111111254eeb25477b68fb85ed929f73a960582,0xe789c5566b53546d46a0af48a4bd3f062d1fefd1,None
4,0x853d955acef822db058eb8505911ed77f175b99e,0x00000000000000000000000000000000000000000000...,0x1042563,0x05eaf873b03fd34df0b5a62bbb88e2ef6fec97837147...,0x4a,0x77e1172245d5b346568bd60c5aa6245a5ffdea7a05fc...,0x43,False,0xe789c5566b53546d46a0af48a4bd3f062d1fefd1,0xa88800cd213da5ae406ce248380802bd53b47647,transfer
5,0x853d955acef822db058eb8505911ed77f175b99e,0x00000000000000000000000000000000000000000000...,0x1042563,0x05eaf873b03fd34df0b5a62bbb88e2ef6fec97837147...,0x4a,0x77e1172245d5b346568bd60c5aa6245a5ffdea7a05fc...,0x44,False,0xa88800cd213da5ae406ce248380802bd53b47647,0x1111111254eeb25477b68fb85ed929f73a960582,None
6,0x853d955acef822db058eb8505911ed77f175b99e,0x00000000000000000000000000000000000000000000...,0x1042563,0x05eaf873b03fd34df0b5a62bbb88e2ef6fec97837147...,0x4a,0x77e1172245d5b346568bd60c5aa6245a5ffdea7a05fc...,0x45,False,0xa88800cd213da5ae406ce248380802bd53b47647,0xe7c6a2767df626bca33e4e86b4f5c0c4fb4cff00,transfer
7,0x853d955acef822db058eb8505911ed77f175b99e,0x00000000000000000000000000000000000000000000...,0x1042563,0x05eaf873b03fd34df0b5a62bbb88e2ef6fec97837147...,0x4a,0x77e1172245d5b346568bd60c5aa6245a5ffdea7a05fc...,0x46,False,0xa88800cd213da5ae406ce248380802bd53b47647,0x1111111254eeb25477b68fb85ed929f73a960582,None


In [85]:
# get the unique token addresses involved in transfers
token_addresses = (
    logs
    .query("transaction_type == 'transfer'")
    ["address"]
    .unique()
)
token_addresses

array(['0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
       '0x853d955acef822db058eb8505911ed77f175b99e'], dtype=object)

In [86]:
defillama_api = DefillamaAPI()
token_data = defillama_api.get_historical_token_prices(token_addresses, df.loc[0].timeStamp)
token_data

{'0x853d955acef822db058eb8505911ed77f175b99e': {'decimals': 18,
  'symbol': 'FRAX',
  'price': 1.001,
  'timestamp': 1681518770,
  'confidence': 0.99},
 '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48': {'decimals': 6,
  'symbol': 'USDC',
  'price': 1,
  'timestamp': 1681518773,
  'confidence': 0.99}}

In [87]:
# process the logs to get the erc20 transactions

def decode_data(data, transaction_type, decimals):
    if decimals is None or decimals is pd.NA:
        decimals = 18
    if transaction_type == "swap":
        # no longer working now that decimals will be nan...
        data = data[2:]
        return [int(data[i:i+64], 16) / 10 ** (decimals) for i in range(0, len(data), 64)]
    else: # assume transfer
        return int(data, 16) / 10 ** (decimals)

erc20_transfers = (
    logs
    .query("transaction_type == 'transfer'")
    .assign(
        decimals=lambda x: x["address"].map(lambda address: token_data.get(address, {}).get("decimals")),
        token_price=lambda x: x["address"].map(lambda address: token_data.get(address, {}).get("price")),
        symbol=lambda x: x["address"].map(lambda address: token_data.get(address, {}).get("symbol")),
    )
    .assign(
        data=lambda x: [decode_data(data, transaction_type, decimals) for data, transaction_type, decimals in zip(x["data"], x["transaction_type"], x["decimals"])]
    )
    .assign(
        usd_value=lambda x: [data * token_price for data, token_price in zip(x["data"], x["token_price"])]
    )
    .drop(columns=["blockHash", "blockNumber", "transactionHash", "removed", "logIndex", "transactionIndex", "address", "transaction_type", "decimals", ])
)

erc20_transfers

,data,from_address,to_address,token_price,symbol,usd_value
0,255.803082,0xe7c6a2767df626bca33e4e86b4f5c0c4fb4cff00,0xe789c5566b53546d46a0af48a4bd3f062d1fefd1,1.000,USDC,255.803082
1,255.863202,0x9a834b70c07c81a9fcd6f22e842bf002fbffbe4d,0xe789c5566b53546d46a0af48a4bd3f062d1fefd1,1.001,FRAX,256.119065
2,255.803082,0xe789c5566b53546d46a0af48a4bd3f062d1fefd1,0x9a834b70c07c81a9fcd6f22e842bf002fbffbe4d,1.000,USDC,255.803082
4,251.976304,0xe789c5566b53546d46a0af48a4bd3f062d1fefd1,0xa88800cd213da5ae406ce248380802bd53b47647,1.001,FRAX,252.228281
6,251.976304,0xa88800cd213da5ae406ce248380802bd53b47647,0xe7c6a2767df626bca33e4e86b4f5c0c4fb4cff00,1.001,FRAX,252.228281


The first transfer informs us of the resolver's wallet address being used to execute the transaction (the to_address: 0x7cea8a6da079dc7501d4eb2bbc937e70a2764cef)
The outflow of the final token (FXS) to the settlement contract (0xa88800cd213da5ae406ce248380802bd53b47647) indicates the final token and the amount paid out.
The balance of the final token is the profit for the resolver.

In [88]:
# simplification and easiest measure of profit is just the difference in usd value of the first entry and the entry to settlement contract

initial_usd_value = erc20_transfers.iloc[0].usd_value
# filter for the transaction where to_address is 0xa88800cd213da5ae406ce248380802bd53b47647
final_usd_value = erc20_transfers.query("to_address == '0xa88800cd213da5ae406ce248380802bd53b47647'").usd_value.sum()
profit = initial_usd_value - final_usd_value # taken - given
profit

3.5748012824163027

## Dashboard Generation

In [89]:
# getting profits for any transaction:


receipts = [etherscan_api.get_transaction_receipt(hash) for hash in df.hash]
logs = [etherscan_api.process_logs(receipt) for receipt in receipts]

profit_volume_list = []

for log, timestamp in zip(logs, df.timeStamp):
    # get the unique token addresses involved in transfers
    token_addresses = (
        log
        .query("transaction_type == 'transfer'")
        ["address"]
        .unique()
    )
    # get the token data for the tokens involved in the transaction
    token_data = defillama_api.get_historical_token_prices(token_addresses, timestamp)
    # process the logs to get the erc20 transactions
    erc20_transfers = (
        log
        .query("transaction_type == 'transfer'")
        .assign(
            decimals=lambda x: x["address"].map(lambda address: token_data.get(address, {}).get("decimals")),
            token_price=lambda x: x["address"].map(lambda address: token_data.get(address, {}).get("price")),
            symbol=lambda x: x["address"].map(lambda address: token_data.get(address, {}).get("symbol")),
        )
        .assign(
            data=lambda x: [decode_data(data, transaction_type, decimals) for data, transaction_type, decimals in zip(x["data"], x["transaction_type"], x["decimals"])]
        )
        .assign(
            usd_value=lambda x: [data * token_price for data, token_price in zip(x["data"], x["token_price"])]
        )
        # .drop(columns=["blockHash", "blockNumber", "transactionHash", "removed", "logIndex", "transactionIndex", "address", "transaction_type", "decimals", ])
    )
    # measure profit (and volume)
    initial_usd_value = erc20_transfers.iloc[0].usd_value
    # filter for the transaction where to_address is 0xa88800cd213da5ae406ce248380802bd53b47647
    final_usd_value = erc20_transfers.query("to_address == '0xa88800cd213da5ae406ce248380802bd53b47647'").usd_value.sum()
    profit = initial_usd_value - final_usd_value # taken - given

    # "true volume" is the sum of only the initial and final; the others are intermediate.
    true_volume = initial_usd_value + final_usd_value

    profit_volume_list.append({"profit": profit, "true_volume": true_volume})

# append to df
df = df.assign(**pd.DataFrame(profit_volume_list))
df.head()
                    

,blockNumber,timeStamp,hash,nonce,blockHash,transactionIndex,from,to,value,gas,gasPrice,isError,txreceipt_status,input,contractAddress,cumulativeGasUsed,gasUsed,confirmations,methodId,functionName,profit,true_volume
0,17048931,1681518767,0x05eaf873b03fd34df0b5a62bbb88e2ef6fec97837147...,26503,0x77e1172245d5b346568bd60c5aa6245a5ffdea7a05fc...,74,0x9108813f22637385228a1c621c1904bbbc50dc25,0xa88800cd213da5ae406ce248380802bd53b47647,0,508565,23544265231,0,1,0x0965d04b000000000000000000000000000000000000...,,3955432,240453,7005,0x0965d04b,settleOrders(bytes data),3.574801,508.031363
1,17048932,1681518779,0xddd47b8c66f61376e14aba9c05e3a7b1aea53e8bf44f...,6097,0x90e4277c73d1e80a05eff9741a0af7f00d7ea307e978...,5,0xcfa62f77920d6383be12c91c71bd403599e1116f,0xa88800cd213da5ae406ce248380802bd53b47647,0,667536,33015544778,0,1,0x0965d04b000000000000000000000000000000000000...,,793292,349212,7004,0x0965d04b,settleOrders(bytes data),-12.155295,21603.886267
2,17048949,1681518983,0x6c83f6076738c8c73daff43c576f57da289d0a798c93...,26684,0x0722b50c4f7539e2539d0ef31f0934a775ea279ac0fe...,6,0xc6c7565644ea1893ad29182f7b6961aab7edfed0,0xa88800cd213da5ae406ce248380802bd53b47647,0,1500000,35557182698,0,1,0x0965d04b000000000000000000000000000000000000...,,1213514,298376,6987,0x0965d04b,settleOrders(bytes data),3.954305,179.366441
4,17048967,1681519199,0xde912cdb7220293b1874623560a48e19debd35529337...,5071,0x322da2b6d1966ae5d7de4a5f0df79e17b2591c9a3d8c...,0,0xee230dd7519bc5d0c9899e8704ffdc80560e8509,0xa88800cd213da5ae406ce248380802bd53b47647,0,2000000,31458663012,0,1,0x0965d04b000000000000000000000000000000000000...,,716163,716163,6969,0x0965d04b,settleOrders(bytes data),-0.463660,20040.463660
5,17049003,1681519679,0x732a2f175ff9d340039ba4b62422c4a9b21d6e36dc5f...,6098,0x81802f635edf069ef1fef18bd47353e126f34b68409c...,19,0xcfa62f77920d6383be12c91c71bd403599e1116f,0xa88800cd213da5ae406ce248380802bd53b47647,0,584404,34324549950,0,1,0x0965d04b000000000000000000000000000000000000...,,2967899,306326,6933,0x0965d04b,settleOrders(bytes data),7.361956,2811.964644


In [90]:
resolver_address_mapping ={
    '0x55dcad916750c19c4ec69d65ff0317767b36ce90': '1inch Labs',
    '0x8acdb3bcc5101b1ba8a5070f003a77a2da376fe8': '1inch Labs',
    '0x84d99aa569d93a9ca187d83734c8c4a519c4e9b1': '1inch Labs',
    '0x3169de0e661d684e0d235f19cf72327173e0be11': '1inch Labs',
    '0x9108813f22637385228a1c621c1904bbbc50dc25': 'Laertes',
    '0x2eb393fbac8aaa16047d4242033a25486e14f345': 'Arctic Bastion',
    '0x7636a5bfd763cefec2da9858c459f2a9b0fe8a6c': 'Arctic Bastion',
    '0xf1b2e1fef70e0383ef29618d02d0dd503ae239ae': 'Arctic Bastion',
    '0x377a1286ff83df266ff11bede2ef600044f3626b': 'Arctic Bastion',
    '0xcfa62f77920d6383be12c91c71bd403599e1116f': 'The Open DAO resolver',
    '0xad7149152a65e6ec97add7b1b1f917dcafcf9b21': 'Seawise',
    '0xd1742b3c4fbb096990c8950fa635aec75b30781a': 'Seawise',
    '0xc975671642534f407ebdcaef2428d355ede16a2c': 'OTEX',
    '0xbd4dbe0cb9136ffb4955ede88ebd5e92222ad09a': 'OTEX',
    '0xc6c7565644ea1893ad29182f7b6961aab7edfed0': 'The T Resolver',
    '0x69313aec23db7e4e8788b942850202bcb6038734': 'Resolver 8',
    '0xee230dd7519bc5d0c9899e8704ffdc80560e8509': 'Kinetex Labs Resolver'
}


In [93]:
# groupby resolvers
(
    df
    .assign(
        transactionCost=lambda x: x["gasUsed"] * x["gasPrice"] / 1e18,
        **{"resolver": lambda x: x["from"].map(lambda address: resolver_address_mapping.get(address, address))},
    )
    .groupby("resolver")
    .agg(
        total_gas_used=("gasUsed", "sum"),
        avg_gas_used=("gasUsed", "mean"),
        avg_gas_price=("gasPrice", "mean"),
        total_transaction_cost=("transactionCost", "sum"),
        avg_transaction_cost=("transactionCost", "mean"),
        total_usd_profit=("profit", "sum"),
        avg_usd_profit=("profit", "mean"),
        total_usd_volume=("true_volume", "sum"),
        avg_usd_volume=("true_volume", "mean"),
    )
)

,total_gas_used,avg_gas_used,avg_gas_price,total_transaction_cost,avg_transaction_cost,total_usd_profit,avg_usd_profit,total_usd_volume,avg_usd_volume
resolver,,,,,,,,,
Resolver 8,9579868,245637.641026,8.070682e+10,0.754300,0.019341,1024.411616,33.045536,8.571909e+05,27651.319125
Laertes,9572638,330090.965517,2.520195e+10,0.239638,0.008263,1581.918151,58.589561,1.288716e+06,47730.232130
The T Resolver,123904408,366581.088757,3.741998e+10,4.591877,0.013585,1849.686432,6.927665,1.339237e+07,50158.702904
The Open DAO resolver,61299995,398051.915584,3.745976e+10,2.331497,0.015140,9308.751220,83.113850,1.159375e+07,103515.619659
Kinetex Labs Resolver,4495725,499525.000000,3.246298e+10,0.143968,0.015996,98.585906,14.083701,2.397380e+05,34248.289512
